# **Description**
---

1. (20%) Please watch the following vidio about correlation and portfolio.(20%)
  - https://www.youtube.com/watch?v=Nu4lHaSh7D4&t=218s


2. (30%) Suppose you have 1 M USD and want to invest in US stock market for 10 years (2012-2022). You will decide the portfolio every three months and buy hold for three months.  At the beginning of every three months: using past 6 months data to compute the risk and correlation of S&P 500 stocks.
  - Select the first stock which has the smallest average of absolute values of correlations with other stocks
  - Select the second stock which has the smallest absolute values of correlation with the first selected one
  - Select the third stock which has the smallest average of absolute values of correlations with the first two selected stocks.
  Repeat until you have 10 stocks. Use variance of return of these ten stocks (computed with last 6 months data) and decide the portfolio in this way:
  $$
  w_i=\frac{\frac{1}{\sigma_i^2}}{\sum_{k=1}^{10}\frac{1}{\sigma_k^2}}
  $$

    That means, if the stock has higher risk , you will invest less.  
After you select 10 stocks and weights are determined, you buy and hold for three month. Then Select again. You repeat these for 10 years.

3. (10%)  Compute the maximum drawdown, annual sharpe ratio of your invesment in these 10 years.

4. (30%) Now we repeat task 2, with risk management. At the beginning of every three month, using **Historical Data**, to decide optimal $\gamma$ and $\delta$. Then in the coming three month, you need to exit if your loss is more than $\gamma$ ATR or your profit is more than $\delta$ ATP. If you exit early, you can take a vocation and wait for the next starting month. If nothing happens, wait until the end of three months. Then repeat these for 10 years. Compute the maximum drawdown, sharpe ratio of your invesment in these 10 years. (20%)

5. (10%) What is your conclusion ? What ideas do you have to improve the performance?

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Task 2 and Task 3:

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from typing import List
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta

In [ ]:
#download different data
def download_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    return yf.download(ticker, start=start, end=end)

In [ ]:
#calculate corr
def calculate_correlation_matrix(df: pd.DataFrame) -> pd.DataFrame:
    return df.pct_change().corr()

In [ ]:
#calculate var
def calculate_variance(df: pd.DataFrame) -> pd.Series:
    return df.pct_change().var()

In [ ]:
#select min one
def select_stocks(correlation_matrix: pd.DataFrame, num_stocks: int) -> List[str]:
    selected_stocks = []
    remaining_stocks = list(correlation_matrix.columns)

    for i in range(num_stocks):
        min_corr_stock = None
        min_corr = float('inf')

        for stock in remaining_stocks:
            if i == 0:  # First stock selection
                avg_corr = correlation_matrix[stock].abs().mean()
            elif i == 1:  # Second stock selection
                avg_corr = abs(correlation_matrix.loc[selected_stocks[0], stock])
            else:
                avg_corr = correlation_matrix.loc[selected_stocks, stock].abs().mean()

            if avg_corr < min_corr:
                min_corr = avg_corr
                min_corr_stock = stock

        selected_stocks.append(min_corr_stock)
        remaining_stocks.remove(min_corr_stock)

    return selected_stocks


In [ ]:
#calculate weight
def calculate_weights(variance: pd.Series, selected_stocks: List[str]) -> pd.Series:
    selected_variances = variance.loc[selected_stocks]
    inverted_variances = 1 / selected_variances
    total = inverted_variances.sum()
    weight=inverted_variances / total
    return weight


In [ ]:
def calculate_max_drawdown(wealth_process: pd.DataFrame) -> float:
    max_drawdown = 0
    peak = wealth_process.iloc[0]['wealth']
    for _, row in wealth_process.iterrows():
        if row['wealth'] > peak:
            peak = row['wealth']
        drawdown = (peak - row['wealth']) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown
    if len(wealth_process) == 0:
        return 0


In [ ]:
def get_wealth_process(selected_stocks: List[str], start: str, end: str, initial: float) -> pd.DataFrame:
    data = {ticker: download_data(ticker, start, end) for ticker in selected_stocks}
    prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in data.items()})
    returns_df = prices_df.pct_change().dropna()

    # Compute wealth process
    weights = calculate_weights(returns_df.var(), selected_stocks)
    wealth_df = (weights * returns_df).sum(axis=1).add(1).cumprod() * initial

    return wealth_df

In [ ]:
def get_sp500_tickers():
    sp500 = yf.Ticker("^GSPC")
    sp500_constituents = sp500.sustainability['constituents'].split(' ')
    return sp500_constituents[:-1] # Remove the last empty element

In [ ]:
!pip install yahoo_fin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 KB 10.7 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=1abdc9c243b3f0e772439ea8795580b0a8ad56a1a58a76e0db995acbd8aa28ba
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24589 sha256=6a6040dfdc8fbd4468344cb06be374c68c730e7afe2194a6b6f236c3951a785b
  Stored in directory: /root/.cache/pip/wheels/d6/9c/58/ee

In [ ]:
#Preparation
risk_free_rate = 0.02
start_date = '2012-01-01'
end_date = '2022-12-31'
initial = 1000000 #total amount
from yahoo_fin.stock_info import *
sp500list=tickers_sp500()
tickers=yf.download("^GSPC",start_date,end_date)
num_stocks = 10
quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')
portfolio_value = initial
max_drawdown = 0
all_returns= []
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')
all_wealth_processes = [] # Define an empty list to store wealth processes


# Download S&P 500 data
ticker = "^GSPC"
sp500_data = download_data(ticker, start_date, end_date)
sp500_prices = sp500_data['Adj Close']
sp500_returns = sp500_prices.pct_change().dropna()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<ipython-input-33-83d4bbf963a1>:10: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  quarters = pd.date_range(start=start_date, end=end_date, freq='Q', closed='right')


In [ ]:
wealth_process = pd.DataFrame({'date': [start_date], 'wealth': [initial]})
wealth_process = wealth_process.set_index('date')

for quarter_end in quarters:
    start = (quarter_end - pd.DateOffset(months=6)).strftime('%Y-%m-%d')
    end = quarter_end.strftime('%Y-%m-%d')

    data = {ticker: download_data(ticker, start, end) for ticker in tickers}
    prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in data.items()})
    correlation_matrix = calculate_correlation_matrix(prices_df)
    variance = calculate_variance(prices_df)

    selected_stocks = select_stocks(correlation_matrix, num_stocks)
    weights = calculate_weights(variance, selected_stocks)

    start_next_quarter = (quarter_end + pd.DateOffset(days=1)).strftime('%Y-%m-%d')
    end_next_quarter = (quarter_end + pd.DateOffset(months=3)).strftime('%Y-%m-%d')

    next_quarter_data = {ticker: download_data(ticker, start_next_quarter, end_next_quarter) for ticker in selected_stocks}
    next_quarter_prices_df = pd.DataFrame({ticker: df['Adj Close'] for ticker, df in next_quarter_data.items()})
    returns = next_quarter_prices_df.pct_change().dropna()

    # Compute daily returns
    daily_returns = (weights * returns).sum(axis=1)
    all_returns.extend(daily_returns)

    # Update wealth process
    end_date = pd.to_datetime(end_next_quarter).date()
    start_date = end_date - pd.DateOffset(months=3) + pd.DateOffset(days=1)

    wealth_period = sp500_returns.loc[start_date:end_date] + 1
    wealth_process_quarter = wealth_period.cumprod() * portfolio_value
    wealth_process_quarter = pd.DataFrame({'date': wealth_process_quarter.index, 'wealth': wealth_process_quarter.values})
    wealth_process_quarter = wealth_process_quarter.set_index('date')
    wealth_process = wealth_process.append(wealth_process_quarter)

    portfolio_value = wealth_process.iloc[-1]['wealth']

# Compute portfolio returns and statistics
portfolio_returns = wealth_process['wealth'].pct_change().fillna(0)
max_drawdown = calculate_max_drawdown(wealth_process)
sharpe_ratio = (np.mean(all_returns) / np.std(all_returns)) * np.sqrt(252)

print(f"Portfolio Value after 10 years: ${wealth_process.iloc[-1]['wealth']:.2f}")
print(f"Maximum Drawdown: {max_drawdown * 100:.2f}%")
print(f"Annual Sharpe Ratio: {sharpe_ratio:.2f}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-51-682726315158>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-51-682726315158>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

<ipython-input-51-682726315158>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-51-682726315158>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

<ipython-input-51-682726315158>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wealth_process = wealth_process.append(wealth_process_quarter)


# Task4:

In [ ]:
import datetime

PRINCIPAL = 1000000
start_str = "2012-01-01"
start_date = datetime.datetime.strptime(start_str, "%Y-%m-%d") - datetime.timedelta(days=180)
end_date = datetime.datetime.strptime(start_str, "%Y-%m-%d")
optimal_gamma=float('inf')
optimal_delta=float('inf')
date_rng = pd.date_range(start='1/1/2012', end='1/1/2022', freq='1d')
PRINCIPAL_daily = pd.DataFrame(index=date_rng,columns=["daily_sum"])
PRINCIPAL_daily_early_exit = pd.DataFrame(index=date_rng,columns=["daily_sum"])


In [ ]:
def trueRange(row):
    high, low, prev_close = row
    return max(high - low, abs(high - prev_close), abs(low - prev_close))

In [ ]:
def calculate_optimal_gammadelta(PRINCIPAL_daily, stock_df, ATR):
    global optimal_gamma, optimal_delta
    max_principal, gammas, deltas = 0, np.arange(2, 3, 0.1), np.arange(3, 4, 0.1)
    for i in gammas:
        for j in deltas:
            exit_principal = 0
            for k in stock_df[0].index:
                exit_principal = PRINCIPAL_daily["daily_sum"][k]
                if PRINCIPAL_daily["daily_return"][k] < -i * ATR or PRINCIPAL_daily["daily_return"][k] > j * ATR:
                    break
            if exit_principal > max_principal:
                max_principal, optimal_gamma, optimal_delta = exit_principal, i, j

In [ ]:
def task4():
  global PRINCIPAL
  sp500list=tickers_sp500()
  sp500_tickers=yf.download("^GSPC",start_date,end_date)

  #get corr
  corr_df = pd.DataFrame(index=sp500_tickers, columns=sp500_tickers)
  for i in sp500_tickers:
      stock_i = yf.download(i, start=start_date, end=end_date)
      for j in sp500_tickers:
          stock_j = yf.download(j, start=start_date, end=end_date-datetime.timedelta(days=1))
          # get the percentage returns
          returns_i = stock_i['Adj Close'].pct_change().dropna()
          returns_j = stock_j['Adj Close'].pct_change().dropna()
          # get the corr
          corr = returns_i.corr(returns_j)
          corr_df.loc[i, j] = corr

  first_stock=corr_df.astype(float).mean(axis=1).idxmin()
  second_stock=corr_df[[first_stock]].astype(float).mean(axis=1).drop(index=[first_stock]).idxmin()
  third_stock=corr_df[[first_stock,second_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock]).idxmin()
  forth_stock=corr_df[[first_stock,second_stock,third_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock]).idxmin()
  fifth_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock]).idxmin()
  sixth_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock,fifth_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock,fifth_stock]).idxmin()
  seventh_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock]).idxmin()
  eighth_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock]).idxmin()
  ninth_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock,eighth_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock,eighth_stock]).idxmin()
  tenth_stock=corr_df[[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock,eighth_stock,ninth_stock]].astype(float).mean(axis=1).drop(index=[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock,eighth_stock,ninth_stock]).idxmin()

  stock_list=[first_stock,second_stock,third_stock,forth_stock,fifth_stock,sixth_stock,seventh_stock,eighth_stock,ninth_stock,tenth_stock]

  Weight=pd.DataFrame(index=stock_list,columns=["variance"])
  # create an empty list
  portfolio_ATR=[]
  for i in stock_list:
    stock_i=yf.download(i,start=start_date,end=end_date-datetime.timedelta(days=1))
    Weight.loc[i]=stock_i["Adj Close"].pct_change().var()
    stock_i["previousClose"]=stock_i["Close"].shift(1)
    stock_i["TR"]=stock_i[["High","Low","previousClose"]].apply(trueRange,axis=1)
    portfolio_ATR.append(stock_i["TR"].mean()/stock_i["Close"][-1])

  Weight["weight"]=(1/Weight["variance"])/(1/Weight["variance"]).sum()

  Weight["shares"]=None
  Weight["wealth_after_three_months"]=None
  stock_df=[]

  for i in stock_list:
    stock_i=yf.download(i,start=end_date,end=end_date+datetime.timedelta(days=91))
    stock_df.append(stock_i)
    Buy_price=stock_i["Open"][0]
    Sell_price=stock_i["Open"][-1]
    Weight.loc[i,"shares"]=PRINCIPAL*Weight["weight"][i]/Buy_price
    Weight.loc[i,"wealth_after_three_months"]=Weight["shares"][i]*Sell_price

  PRINCIPAL=Weight["wealth_after_three_months"].sum()

  # get the daily return
  for i in stock_df[0].index:
    PRINCIPAL_sum = 0
    for j in range(10):
      PRINCIPAL_sum += stock_df[j]["Open"][i]*Weight.loc[stock_list[j],"shares"]
    PRINCIPAL_daily.loc[i]=PRINCIPAL_sum
  PRINCIPAL_daily["daily_return"]=PRINCIPAL_daily["daily_sum"].pct_change()
  # get ATR
  ATR = 0
  for i in range(10):
    ATR += Weight["weight"][i]*portfolio_ATR[i]
   # update
  for i in stock_df[0].index:
    PRINCIPAL_daily_early_exit.loc[i]=PRINCIPAL_daily["daily_sum"][i]
    if (PRINCIPAL_daily["daily_return"][i]<-optimal_gamma*ATR or PRINCIPAL_daily["daily_return"][i]>optimal_delta*ATR) and i != stock_df[0].index[0]:
      PRINCIPAL = PRINCIPAL_daily["daily_sum"][i]
      break
  calculate_optimal_gammadelta(PRINCIPAL_daily,stock_df,ATR)



In [ ]:
for i in range(40):
  task4()
  print(optimal_gamma,optimal_delta)
  start_date=start_date+datetime.timedelta(days=91)
  end_date=end_date+datetime.timedelta(days=91)

流式输出内容被截断，只能显示最后 5000 行内容。
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[******

In [ ]:
#money I get
PRINCIPAL

13739778.213049887

In [ ]:
# calculate the daily_return and max_drawdown for every day
PRINCIPAL_daily_early_exit["daily_return"]=PRINCIPAL_daily_early_exit["daily_sum"].pct_change()
PRINCIPAL_daily_early_exit["max_drawdown"]=PRINCIPAL_daily_early_exit["daily_sum"]/PRINCIPAL_daily_early_exit["daily_sum"].expanding().max()-1

In [ ]:
#Max_Drawdown
PRINCIPAL_daily_early_exit=PRINCIPAL_daily_early_exit.dropna(subset="daily_sum")
Max_Drawdown=PRINCIPAL_daily_early_exit["max_drawdown"].min()
Max_Drawdown

-0.19180321267509204

In [ ]:
#sharpe ratio
sharpe_ratio=math.sqrt(252)*PRINCIPAL_daily_early_exit["daily_return"].mean()/PRINCIPAL_daily_early_exit["daily_return"].std()
sharpe_ratio

1.8236848635057228

# Task5:

Conclusion: Risk Management matters a lot.

Improvement:
1.	Parameter Optimization: Perform a thorough optimization of the parameters used in the trading strategy, such as the risk parameters (𝛾 and 𝛿), the number of stocks in the portfolio, and the rolling window size for calculating ATR, variance, and correlation. This can be achieved using methods like grid search, random search, or more advanced optimization techniques like genetic algorithms or Bayesian optimization.
2.	Machine Learning Techniques: Utilize machine learning techniques to predict stock returns, correlations, or other relevant variables that can influence the selection of stocks or the weighting scheme. Techniques like regression models, decision trees, or deep learning can be applied.
3.	Alternative Portfolio Optimization Techniques: Instead of using a risk-based weighting scheme, consider using other portfolio optimization techniques such as Modern Portfolio Theory (MPT) or the Black-Litterman model. These methods aim to maximize the risk-adjusted return of the portfolio by considering both the expected returns and the covariance of asset returns.
4.	Stop-loss and Profit-taking Orders: Implement dynamic stop-loss and profit-taking orders to automatically exit positions when predefined risk or profit levels are reached. This can help protect the portfolio from sudden market movements and lock in profits.
5.	Risk Management: Continuously monitor and manage risk by adjusting position sizes based on the volatility of the market or individual assets. Implementing a risk parity approach could be another option to ensure equal risk contribution from all assets in the portfolio.

